In [4]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
res = requests.get('http://www.commoditiescontrol.com/live-cotton-price.html')
html = res.text
soup = BeautifulSoup(html,'html.parser')
div = soup.find_all('div',{'class':'data_tbl'})
trs = div[1].find_all('tr')
print(trs[3].find('td').text)
Live_cotton_Price = int(trs[3].find('td').text)

In [ ]:
dataset = pd.read_excel('Book1.xlsx')
print(dataset)

In [7]:
X = dataset.iloc[:, [1,2,3,4,5,6,7,8,9,10]].values
y = dataset.iloc[:, [0]].values

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [9]:
df=pd.DataFrame(X)
#print(df)
df.to_csv(r'data.csv', index = False)

In [10]:
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[3])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [11]:
df=pd.DataFrame(X)
#print(df)
df.to_csv(r'data1.csv', index = False)

In [12]:
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[11])],remainder='passthrough')
X=np.array(ct.fit_transform(X))
df=pd.DataFrame(X)
#print(df)
df.to_csv(r'data2.csv', index = False)

In [13]:
y = y.reshape(len(y),1)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [15]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)

In [16]:
# # Applying PCA
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 2)
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)
# explained_variance = pca.explained_variance_ratio_
# print(explained_variance)

## REGRESSION

### Polynomial

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(X_train)
regressor1 = LinearRegression()
regressor1.fit(X_poly, y_train)

In [ ]:
y_pred1 = sc_y.inverse_transform(regressor1.predict(poly_reg.transform(sc_X.transform(X_test))))
for ele in range(len(y_pred1)):
  if(y_pred1[ele]<0):
    y_pred1[ele] = 0
# np.set_printoptions(precision=2)
print(np.concatenate((y_pred1.reshape(len(y_pred1),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred1)

## SVM

In [ ]:
from sklearn.svm import SVR
regressor2 = SVR(kernel = 'rbf')
regressor2.fit(X_train, y_train)

In [ ]:
y_pred2 = sc_y.inverse_transform(regressor2.predict(sc_X.transform(X_test)))
for ele in range(len(y_pred2)):
  if(y_pred2[ele]<0):
    y_pred2[ele] = 0
np.set_printoptions(precision=2)
print(np.concatenate((y_pred2.reshape(len(y_pred2),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import r2_score
import sklearn.metrics as metrics
print('r2_score:',r2_score(y_test, y_pred2))
print('Mean abolute error is:', metrics.mean_absolute_error(y_test,y_pred2))
print('Mean squared error is:', metrics.mean_squared_error(y_test,y_pred2))
print('Root mean squared error is:', np.sqrt(metrics.mean_squared_error(y_test,y_pred2)))

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor3 = DecisionTreeRegressor(random_state = 0)
regressor3.fit(X_train, y_train)

In [ ]:
y_pred3 = sc_y.inverse_transform(regressor3.predict(sc_X.transform(X_test)))
# np.set_printoptions(precision=2)
for ele in range(len(y_pred3)):
  if(y_pred3[ele]<0):
    y_pred3[ele] = 0
print(np.concatenate((y_pred3.reshape(len(y_pred3),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred3)

# Random Forrest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor4 = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor4.fit(X_train, y_train)

In [ ]:
y_pred4 = sc_y.inverse_transform(regressor4.predict(sc_X.transform(X_test)))
np.set_printoptions(precision=2)
for ele in range(len(y_pred4)):
  if(y_pred4[ele]<0):
    y_pred4[ele] = 0
print(np.concatenate((y_pred4.reshape(len(y_pred4),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred4)

# Boosting

In [ ]:
from xgboost import XGBRegressor
regressor5 = XGBRegressor()
regressor5.fit(X_train, y_train)
y_pred5 = sc_y.inverse_transform(regressor5.predict(sc_X.transform(X_test)))
for ele in range(len(y_pred5)):
  if(y_pred5[ele]<0):
    y_pred5[ele] = 0

In [ ]:
np.set_printoptions(precision=2)
print(np.concatenate((y_pred5.reshape(len(y_pred5),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred5)

# Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor4, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

#Predictng Results

In [ ]:
C =  30000.0
I =  0.01
D =  0.05
K =  0.8
  
EIL = C/ (Live_cotton_Price * I * D * K)
print(EIL)

## Taking a sample data

In [ ]:
X_predict = [[1.0,	0.0,	0.0,	1.0,	0.0,	0.0,	0.0,	1.0,	0.0,	0.0,	0.0,	0.0,	0.0,	0.0,	31.38571428571429,	22.585714285714285,	62.4,	84.71428571428571,	63.142857142857146,	0.0,	0.0]]
X_predictSC = sc_X.fit_transform(X_predict)
y_predict = sc_y.inverse_transform(regressor4.predict(X_predictSC))
print(y_predict*1000)

In [ ]:
if y_predict*100>EIL:
  print('Crop has crossed EIL, start preventive measures')
else:
  print('Crop has not crossed EIL')